In [ ]:
from scipy.io import netcdf
import numpy as np
from pylab import clf, plot, show, floor, ceil, imshow
import matplotlib
import matplotlib.pyplot as plt 
%matplotlib inline
#xlabel, ylabel, legend, savefig, colorbar, title, clim, pcolor, cm, contourf
import os
import csv
import sys
import glob
from netCDF4 import Dataset

from IPython.display import display, Math, Latex, Image
from mpl_toolkits.basemap import Basemap

sys.path.append('/noc/users/am8e13/PyNGL-1.4.0/lib/python2.7/site-packages/PyNGL/')
import Ngl
sys.path.append('/noc/users/am8e13/Python/')
import komod
sys.path.append('/noc/users/am8e13/Python/PyNIO-1.4.0/')
import Nio

In [ ]:
f = Dataset('/scratch/general/am8e13/NEMO_data/ORCA0083-N01_1980to1982y03T.nc')

In [ ]:
# Getting NEMO data
file2read = Dataset("/scratch/general/am8e13/NEMO_data/mask.nc")
tmask=file2read.variables['tmask']
tmask=tmask[:]*1
file2read = Dataset("/scratch/general/am8e13/NEMO_data/ORCA0083-N01_1983to1992y10T.nc")
T_nemo=file2read.variables['votemper']
T_nemo=T_nemo[:]*1
S_nemo=file2read.variables['vosaline']
S_nemo=S_nemo[:]*1
lon_nemoo=file2read.variables['nav_lon']
lon_nemoo=lon_nemoo[:]*1
lat_nemoo=file2read.variables['nav_lat']
lat_nemoo=lat_nemoo[:]*1
depth_nemo=file2read.variables['deptht']
depth_nemo=depth_nemo[:]*1

T_nemo[tmask == 0] = np.nan
T_sq = T_nemo.squeeze(axis=0)
T_nemo = np.array(T_sq)
T_nemo = T_nemo[:,700:,:]

S_nemo[tmask == 0] = np.nan
S_sq = S_nemo.squeeze(axis=0)
S_nemo = np.array(S_sq)
S_nemo = S_nemo[:,700:,:]

lat_nemo = np.array(lat_nemoo[700:,:])
lon_nemo = np.array(lon_nemoo[700:,:])

In [ ]:
file2read = netcdf.NetCDFFile("/scratch/general/am8e13/noaa_climatology/s00_04.nc",'r')
s_oa=file2read.variables['s_oa']
s_oa=s_oa[:]*1
s_mn=file2read.variables['s_mn']
s_mn=s_mn[:]*1
s_an=file2read.variables['s_an']
s_an=s_an[:]*1
S_noaa=file2read.variables['s_an']
S_noaa=S_noaa[:]*1
file2read = netcdf.NetCDFFile("/scratch/general/am8e13/noaa_climatology/t00_04.nc",'r')
t_oa=file2read.variables['t_oa']
t_oa=t_oa[:]*1
t_mn=file2read.variables['t_mn']
t_mn=t_mn[:]*1
t_an=file2read.variables['t_an']
t_an=t_an[:]*1
T_noaa=file2read.variables['t_an']
T_noaa=T_noaa[:]*1
lon=file2read.variables['lon']
lon=lon[:]*1
lat=file2read.variables['lat']
lat=lat[:]*1
depth=file2read.variables['depth']
depth=depth[:]*1

lon_noaa=file2read.variables['lon']
lon_noaa=lon_noaa[:]*1
lat_noaa=file2read.variables['lat']
lat_noaa=lat_noaa[:]*1
depth_noaa=file2read.variables['depth']
depth_noaa=depth_noaa[:]*1

In [ ]:
# Arctic Transects
# Barents Strait
barents = [77,-20,79,10]
# Fram Strait
fram = [80,-13,79,10]
# Bering Strait

#Davis Strait

# Arctic 1
arctic1 = [83,-28,74,136]
# Arctic 2
arctic2 = [71,-139,75,53]
# Arctic 3
arctic3 = [78,-115,76,111]
# Arctic 4
arctic4 = [60,180,60,0]
# St Anna Strait
# 

In [ ]:
def comp_plot(data, region):
    import matplotlib as mpl
    # This function plots time evolution of temperature/salinity on a transect.
    # Transect are: Fram Strait, Bering Strait ...
    lat1 = region[0]
    lon1 = region[1]
    lat2 = region[2]
    lon2 = region[3]
    fig, axes = plt.subplots(2,4,sharex='col', sharey='row') 
    ind = 0

    tempbounds = [-2,-1,0,1,2,3,4,5,6,7,8,9,10,11,12]
    saltbounds = [28,30,33,34,35,36,37,38]
    for run in data:
        data_prof, x_kilometers, m_grid, n_grid  = \
        get_transect1(lat_data[run], lon_data[run], T_data[run], lat1, lon1, lat2, lon2,\
                                                                 npoints = 50, pdif = 1, norep=False)   
        Z,z2 = lastnan(data_prof,Z_data[run])
        if run == 'run36km':
            imT = axes.flat[ind].contourf(x_kilometers,Z,data_prof[0:z2,:],vmin=-2,vmax=10,levels = tempbounds,\
                    extend = 'both')
        else:
            axes.flat[ind].contourf(x_kilometers,Z,data_prof[0:z2,:],vmin=-2,vmax=10,levels = tempbounds,\
                    extend = 'both')
        axes.flat[ind].contour(x_kilometers,Z,data_prof[0:z2,:],colors='k',levels = tempbounds,\
                    extend = 'both')
        axes.flat[ind].set_title("Temperature " + titles_data[run])
        axes.flat[ind].title.set_fontsize('14')
        if ind == 0:
            axes.flat[ind].set_ylabel('m')
        #axes.flat[ind].set_xlabel('km')
        
        data_prof, x_kilometers, m_grid, n_grid  = \
        get_transect1(lat_data[run], lon_data[run], S_data[run], lat1, lon1, lat2, lon2,\
                                                                 npoints = 50, pdif = 1, norep=False)   

        if run == 'run36km':
            imS = axes.flat[ind+4].contourf(x_kilometers,Z,data_prof[0:z2,:],vmin=28,vmax=38,levels = saltbounds,\
                    extend = 'both')
        else:
            axes.flat[ind+4].contourf(x_kilometers,Z,data_prof[0:z2,:],vmin=28,vmax=38,levels = saltbounds,\
                    extend = 'both')
        axes.flat[ind+4].contour(x_kilometers,Z,data_prof[0:z2,:],colors='k',levels = saltbounds,\
                    extend = 'both')
        axes.flat[ind+4].set_title("Salinity " + titles_data[run])
        axes.flat[ind+4].title.set_fontsize('14')
        if ind == 0:
            axes.flat[ind+4].set_ylabel('m')
        axes.flat[ind+4].set_xlabel('km')
        
        
        #for item in ([axes.flat[ind].title, axes.flat[ind].xaxis.label, axes.flat[ind].yaxis.label]):
         #   item.set_fontsize(14)
        ind = ind + 1
    
    cbar_ax = fig.add_axes([2.15, 0.2, 0.045, 0.7])
    cbar = plt.colorbar(imS, cax=cbar_ax,)
    cbar.ax.set_ylabel('psu')  
    
    cbar_ax2 = fig.add_axes([2.15, 1.2, 0.045, 0.7])
    cbar2 = plt.colorbar(imT, cax=cbar_ax2)
    cbar2.ax.set_ylabel('C')  
    
    fig.subplots_adjust(right=2.1,top=2.)